In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"/workspaces/Supply-Chain-Management/Data/supplier_contracts_dataset.csv")
df = df.replace({np.nan: None})
# Rename columns to remove spaces
# df.columns = [col.replace(' ', '_') for col in df.columns]

In [3]:
# df.columns = df.columns.str.lower()
# df.to_csv(r"/workspaces/Supply-Chain-Management/Data/supplier_contracts_dataset.csv",index=False)

In [4]:
df.columns

Index(['supplier_id', 'supplier_name', 'supplier_type', 'risk_level',
       'compliance_issues', 'key_terms', 'past_performance',
       'negotiate_recommendation', 'supply_chain_disruption',
       'quality_metrics', 'cost_metrics'],
      dtype='object')

In [5]:

df = df.rename(columns={'supplier_id':'id'})
# # Convert only the relevant text-based fields to string
text_fields = ['supplier_name', 'supplier_type', 'risk_level',
       'compliance_issues', 'key_terms', 'past_performance',
       'negotiate_recommendation', 'supply_chain_disruption',
       'quality_metrics', 'cost_metrics']

# Ensure the specified text fields are strings
for field in text_fields:
    df[field] = df[field].astype(str)

**MINSEARCH**

In [6]:
# df.columns = df.columns.str.lower()
# df.to_csv(r"/workspaces/Supply-Chain-Management/Data/supplier_contracts_dataset.csv",index=False)

In [7]:
documents = df.to_dict(orient='records')

Get top 10 list of high risk level contracts

In [8]:
import minsearch

# Create an index
index = minsearch.Index(
    text_fields=text_fields,
    keyword_fields=[]
)

# Fit the index with the documents
index.fit(documents)

# Example search query
query = "high risk level"
results = index.search(query, num_results=10)

# Print results
for result in results:
    print(result)

{'id': 'S2482', 'supplier_name': 'Supplier 2482', 'supplier_type': 'Service Provider', 'risk_level': 'High', 'compliance_issues': 'Non-Compliance with Standards', 'key_terms': '45-day payment, 10-day delivery', 'past_performance': 'Poor', 'negotiate_recommendation': 'Include penalty clauses for late delivery, Include compliance monitoring, Adjust delivery schedules', 'supply_chain_disruption': 'No', 'quality_metrics': '2.99% defect rate, Meets standards', 'cost_metrics': '$60.18/unit, $5539.99 total cost'}
{'id': 'S2481', 'supplier_name': 'Supplier 2481', 'supplier_type': 'Manufacturer', 'risk_level': 'High', 'compliance_issues': 'Substandard Quality', 'key_terms': '30-day payment, 5-day delivery', 'past_performance': 'Good', 'negotiate_recommendation': 'Include penalty clauses for late delivery, Seek alternative suppliers, Adjust delivery schedules', 'supply_chain_disruption': 'No', 'quality_metrics': '1.04% defect rate, Meets standards', 'cost_metrics': '$66.06/unit, $5158.25 total c

Get the Contract types that has high risk and their count

In [9]:
import minsearch
from collections import Counter
# Create an index
index = minsearch.Index(
    text_fields=text_fields,
    keyword_fields=['risk_level']
)

# Fit the index with the documents
index.fit(documents)

# Perform the search for high-risk level contracts
filter_dict = {'risk_level': 'High'}
results = index.search(query='high risk level', filter_dict=filter_dict, num_results=len(documents))

# Extract and print contract types with high risk level
high_risk_contract_types = [result['supplier_type'] for result in results]

# Count the occurrences of each contract type
contract_type_counts = Counter(high_risk_contract_types)

# Print the count of each contract type
print("Count of each supplier type with high risk level:")
for contract_type, count in contract_type_counts.items():
    print(f"{contract_type}: {count}")

Count of each supplier type with high risk level:
Retailer: 197
Distributor: 198
Manufacturer: 201
Service Provider: 219


**GROQ API**

In [10]:
# from openai import OpenAI

# client = OpenAI()

In [11]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [12]:
df.columns

Index(['id', 'supplier_name', 'supplier_type', 'risk_level',
       'compliance_issues', 'key_terms', 'past_performance',
       'negotiate_recommendation', 'supply_chain_disruption',
       'quality_metrics', 'cost_metrics'],
      dtype='object')

In [13]:

# Function to call the LLM (Groq API)
# def llm(prompt, model='gpt-4o-mini'):
#     # Assuming `client` is the Groq API client instance
#     response = client.chat.completions.create(
#         model=model,
#         messages=[{"role": "user", "content": prompt}]
#     )
    
#     return response.choices[0].message.content



In [14]:
df.columns


Index(['id', 'supplier_name', 'supplier_type', 'risk_level',
       'compliance_issues', 'key_terms', 'past_performance',
       'negotiate_recommendation', 'supply_chain_disruption',
       'quality_metrics', 'cost_metrics'],
      dtype='object')

In [15]:

# Function to perform the search on supplier contracts based on query

def search(query, filter_dict=None, max_results=10):
    # Filter the DataFrame based on risk level (if provided)
    if filter_dict:
        filtered_df = df[df['risk_level'] == filter_dict.get('risk_level', '')]
    else:
        filtered_df = df
    # Convert the filtered data to a list of dictionaries and limit the number of results
    results = filtered_df.to_dict(orient='records')[:max_results]
    return results

# Function to build a clearer prompt for Groq API
def build_clear_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context += (
            f"- **Supplier_Type**: {doc['supplier_type']}\n"
            f"  **Supplier_Name**: {doc['supplier_name']}\n"
            f"  **Risk_Level**: {doc['risk_level']}\n"
            f"  **Compliance_Issues**: {doc['compliance_issues']}\n"
            f"  **Key_Terms**: {doc['key_terms']}\n"
            f"  **Negotiate_Recommendation**: {doc['negotiate_recommendation']}\n"
            f"  **Quality_Metrics**: {doc['quality_metrics']}\n"
            f"  **Past_Performance**: {doc['past_performance']}\n"
            f"  **Supply_Chain_Disruption**: {doc['supply_chain_disruption']}\n"
            f"  **Cost_Metrics**: {doc['cost_metrics']}\n\n"
        )
    
    prompt = (
        f"QUESTION: {query}\n\n"
        f"CONTEXT:\n{context}"
    )
    
    return prompt

# Function to call the LLM (Groq API)
def llm(prompt, model='Llama3-groq-70b-8192-tool-use-preview'):
    # Assuming client is the Groq API client instance
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=model
    )
    
    return response.choices[0].message.content

# Function to perform the full RAG (Retrieve and Generate) process
def rag(query, model='Llama3-groq-70b-8192-tool-use-preview'):
    # Search for high-risk contracts (you can modify filter_dict based on needs)
    search_results = search(query, filter_dict={'risk_level': 'High'})
    
    # Build the prompt using the search results
    prompt = build_clear_prompt(query, search_results)
    
    # Get the LLM response based on the prompt
    answer = llm(prompt, model=model)
    
    return answer

# Example usage
question = "Give supplier types, quality metrics, supply chain disruptions, and their negotiation recommendations for high-risk contracts"
answer = rag(question)
print(answer)

Supplier Types: Manufacturer, Retailer, Distributor, Service Provider
Quality Metrics: Defect rates, standards compliance
Supply Chain Disruptions: Yes, No
Negotiation Recommendations: Seek alternative suppliers, Include penalty clauses for late delivery, Adjust delivery schedules, Include compliance monitoring


In [16]:
print("\n## Risk-Based Queries")

question = "Which suppliers have the most non-compliance issues, regardless of risk level?"
answer = rag(question)
print(answer)

print("\n## Compliance & Legal Queries:")

question = "Identify suppliers where compliance monitoring is recommended."
answer = rag(question)
print(answer)

print("\n## Cost and Financial Metrics:")

question = "List suppliers that offer the best cost metrics but are classified as high risk."
answer = rag(question)
print(answer)

print("\n## Contractual Terms and Recommendations:")

question = "Which suppliers have penalty clauses for late delivery in their contracts, and what are the associated risks?"
answer = rag(question)
print(answer)

print("\n## Supplier Relationship Queries:")

question = "What are the relationship metrics for suppliers with the best past performance scores?"
answer = rag(question)
print(answer)

print("\n## Opportunity and Innovation Queries:")

question = "Identify suppliers with innovative solutions despite having poor quality metrics."
answer = rag(question)
print(answer)

print("\n# Custom Queries")

question = "Show me all suppliers with a combination of poor quality metrics, high compliance issues, and good past performance scores."
answer = rag(question)
print(answer)

print("\n")

question = "What are the patterns between compliance issues and cost metrics in supplier contracts?"
answer = rag(question)
print(answer)



## Risk-Based Queries
Based on the provided data, the suppliers with the most non-compliance issues, regardless of risk level, are:

1. Supplier 6 (Manufacturer) - Non-Compliance with Standards
2. Supplier 8 (Retailer) - Non-Compliance with Standards
3. Supplier 12 (Distributor) - Substandard Quality
4. Supplier 14 (Service Provider) - Non-Compliance with Standards
5. Supplier 15 (Distributor) - Non-Compliance with Standards
6. Supplier 26 (Distributor) - Late Delivery
7. Supplier 28 (Retailer) - Late Delivery
8. Supplier 32 (Retailer) - Non-Compliance with Standards
9. Supplier 38 (Retailer) - Late Delivery
10. Supplier 40 (Retailer) - Non-Compliance with Standards

## Compliance & Legal Queries:
Based on the provided data, the suppliers where compliance monitoring is recommended are:

1. Supplier 12 (Distributor) - High risk level, substandard quality
2. Supplier 40 (Retailer) - High risk level, non-compliance with standards

## Cost and Financial Metrics:
Supplier 6, Supplier 8, Su

**Retrieval Evaluation**

In [17]:
from tqdm.auto import tqdm

In [18]:
df_question = pd.read_csv(r'/workspaces/Supply-Chain-Management/Data/ground-truth-retrieval.csv')

In [19]:
ground_truth = df_question.to_dict(orient='records')

In [20]:
ground_truth

[{'id': 'S0001',
  'question': 'What is the risk level of Supplier 1 and what compliance issues have they had?'},
 {'id': 'S0001',
  'question': 'What are the key terms of the contract with Supplier 1?'},
 {'id': 'S0001',
  'question': 'How has Supplier 1 performed in the past and what is the recommendation for negotiation?'},
 {'id': 'S0001',
  'question': 'Has there been any supply chain disruption with Supplier 1 and what are the quality metrics?'},
 {'id': 'S0001',
  'question': 'What are the cost metrics for Supplier 1 and how can we adjust the delivery schedules?'},
 {'id': 'S0002', 'question': 'What is the risk level of Supplier 2?'},
 {'id': 'S0002',
  'question': 'What are the key terms of the contract with Supplier 2?'},
 {'id': 'S0002',
  'question': 'What is the recommendation for negotiation with Supplier 2?'},
 {'id': 'S0002',
  'question': 'What is the defect rate of the products supplied by Supplier 2?'},
 {'id': 'S0002',
  'question': 'What is the total cost of the pro

In [21]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/12500 [00:00<?, ?it/s]

{'hit_rate': 0.86384, 'mrr': 0.8502181587301586}

In [22]:
ground_truth

[{'id': 'S0001',
  'question': 'What is the risk level of Supplier 1 and what compliance issues have they had?'},
 {'id': 'S0001',
  'question': 'What are the key terms of the contract with Supplier 1?'},
 {'id': 'S0001',
  'question': 'How has Supplier 1 performed in the past and what is the recommendation for negotiation?'},
 {'id': 'S0001',
  'question': 'Has there been any supply chain disruption with Supplier 1 and what are the quality metrics?'},
 {'id': 'S0001',
  'question': 'What are the cost metrics for Supplier 1 and how can we adjust the delivery schedules?'},
 {'id': 'S0002', 'question': 'What is the risk level of Supplier 2?'},
 {'id': 'S0002',
  'question': 'What are the key terms of the contract with Supplier 2?'},
 {'id': 'S0002',
  'question': 'What is the recommendation for negotiation with Supplier 2?'},
 {'id': 'S0002',
  'question': 'What is the defect rate of the products supplied by Supplier 2?'},
 {'id': 'S0002',
  'question': 'What is the total cost of the pro

Finding the best paramters

In [23]:
df_validation = df_question[:100]
df_test = df_question[100:]

In [24]:
gt_val = df_validation.to_dict(orient='records')

In [26]:
import random
# Define the parameter ranges for optimization
param_ranges = {
    'Supplier Name': (0.0, 3.0),
    'Supplier Type': (0.0, 3.0),
    'Risk Level': (0.0, 3.0),
    'Compliance Issues': (0.0, 3.0),
    'Key Terms': (0.0, 3.0),
    'Past Performance': (0.0, 3.0),
    'Negotiate Recommendation': (0.0, 3.0),
    'Supply Chain Disruption': (0.0, 3.0),
    'Quality Metrics': (0.0, 3.0),
    'Cost Metrics': (0.0, 3.0)
}

# Optimization function
def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Use float('-inf') if maximizing

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

# Placeholder for the search function
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    # Dummy search implementation
    # Replace with actual search logic
    results = [{'Supplier Name': query, 'Score': random.uniform(0, 1)} for _ in range(10)]
    return results

# Objective function
def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['Supplier Name'], boost_params)

    # Dummy evaluation function
    # Replace with actual evaluation logic
    def evaluate(gt_val, search_function):
        mrr = random.uniform(0, 1)  # Dummy MRR score
        return {'mrr': mrr}

    results = evaluate(gt_val, search_function)
    return results['mrr']

# Optimize the parameters
best_params, best_score = simple_optimize(param_ranges, objective, n_iterations=20)
print(f'Best Parameters: {best_params}')
print(f'Best Score: {best_score}')

# Improved search function with optimized parameters
def minsearch_improved(query):
    boost = best_params

    results = minsearch_search(query, boost)
    return results

# Evaluate the final search function
# Dummy evaluation function
def evaluate(ground_truth, search_function):
    mrr = random.uniform(0, 1)  # Dummy MRR score
    return {'mrr': mrr}

evaluate(df_test.to_dict(orient='records'), lambda q: minsearch_improved(q['Supplier Name']))

Best Parameters: {'Supplier Name': 1.8651033748686987, 'Supplier Type': 2.5373938642674743, 'Risk Level': 1.4398958990043025, 'Compliance Issues': 2.839650601427819, 'Key Terms': 1.145606076461828, 'Past Performance': 1.9339512227226145, 'Negotiate Recommendation': 0.22360354984971453, 'Supply Chain Disruption': 2.3231146958764777, 'Quality Metrics': 0.18269751425405978, 'Cost Metrics': 2.9674884288717056}
Best Score: 0.981027859548881


{'mrr': 0.7054360687214983}

**RAG evaluation**

In [27]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [28]:
len(ground_truth)

12500

In [29]:
record = ground_truth[0]

NameError: name 'answer_llm' is not defined